In [3]:
from transformers import BertTokenizer, TFBertModel
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import pandas as pd

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from sklearn.utils import class_weight

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [5]:
def get_bert_encodings(text):
    tokenized_output = bert_tokenizer([text],
        max_length=20,
        truncation=True,
        padding='max_length', 
        return_tensors='tf')
    
    bert_encoding = bert_model(
        tokenized_output
    )
    
    return bert_encoding
test_output = get_bert_encodings("TYLER WAS HERE")

In [15]:
test_output[1].numpy()[0]

array([-0.6857177 ,  0.46915683,  0.9997829 , -0.9947774 ,  0.9485398 ,
        0.9728091 ,  0.97654307, -0.99891067, -0.98978007, -0.5543709 ,
        0.9898439 ,  0.99892724, -0.99975926, -0.9997593 ,  0.9173126 ,
       -0.96885884,  0.9921755 , -0.5221493 , -0.999954  , -0.7563719 ,
       -0.7913862 , -0.99986094,  0.10189829,  0.9938236 ,  0.9606563 ,
        0.06469826,  0.9899247 ,  0.9999591 ,  0.90349084, -0.5065529 ,
        0.2290757 , -0.99254704,  0.93870676, -0.9989391 ,  0.03679116,
        0.4145831 ,  0.82479745, -0.18310285,  0.57812655, -0.9635514 ,
       -0.6804126 , -0.8242758 ,  0.7453289 , -0.4536972 ,  0.9879943 ,
        0.2476247 , -0.06452269,  0.0553384 , -0.06375622,  0.9979952 ,
       -0.9664963 ,  0.89600295, -0.99913174,  0.97571707,  0.99765074,
        0.3664212 ,  0.9972112 ,  0.14247909, -0.99976057, -0.13278589,
        0.96648866,  0.2573793 ,  0.93509686,  0.03741625,  0.6031064 ,
       -0.37438875, -0.9391496 ,  0.10242832, -0.44210544, -0.03

In [12]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

# https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf

[nltk_data] Downloading package punkt to /home/tychi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tychi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tychi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tychi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/tychi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
